## Distributed geospatial

In this section, we'll learn how to use a cluster of machines to create a cloud-free composite. Because of the amount of data involved, we'll distribute the workload on a cluster of machines.


As a reminder, Dask works by using high-level APIs (like xarray) to build task graphs. These task graphs are then executed by a scheduler:

![](https://docs.dask.org/en/latest/_images/dask-overview.svg)

Ideally, we don't need to change our use of high-level APIs like xarray or Dask DataFrame at all. We just swap out a distributed scheduler / cluster for the single-machine scheduler.

That said, it's worth remembering that going distributed makes many things more challenging. You need to think more about how to start workers, communication, distributed filesystems, environments, .... If you don't *need* a cluster, then you should avoid it. But for large enough problems, it's nice to have the option.

## Create a cluster

This JupyterHub is configured with [Dask Gateway](https://gateway.dask.org/), a convenient way to create Dask clusters.

In [ ]:
import dask_gateway

cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.scale(16)

cluster

Since we created a Client connected to that cluster, any operations using Dask will use that cluster. 

## Access Earth systems science data

We'll work with [monthly Daymet for North America](https://planetarycomputer.microsoft.com/dataset/daymet-monthly-na) data, providing summaries of temperature, precipitation, and other variables.

In [ ]:
import pystac_client

stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")
c = stac.get_collection("daymet-monthly-na")
c

The data is provided in Zarr format.

In [ ]:
asset = c.assets["zarr-https"]
asset

Which can be loaded into xarray.

In [ ]:
import xarray as xr

ds = xr.open_zarr(asset.href, **asset.extra_fields["xarray:open_kwargs"])
ds

In [ ]:
x = ds["prcp"].sel(time=slice("1985", None)).mean(dim="time").compute()

In [ ]:
import hvplot.xarray

x.hvplot.image(x="x", y="y", rasterize=True, width=900, height=500)

In [ ]:
ts = ds["tmax"].mean(dim=("y", "x")).compute()

In [ ]:
ts.plot(figsize=(12, 6));

In [ ]:
ts.groupby("time.year").mean().plot();

## Access remote sensing data

Now we'll make a cloudless composite from a bunch of setallite imagery.

In [ ]:
import pystac_client

bbox = [-59.69, -34.74, -58.24, -33.65]
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bbox,
    datetime="2018-01-01/2020-12-31",
    collections=["sentinel-2-l2a"],
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"lt": 25}},
)

In [ ]:
%%time
items = list(search.get_all_items())
print(len(items))

In [ ]:
import planetary_computer

signed_items = []
for item in items:
    item = item.clone()
    item.clear_links()
    signed_items.append(planetary_computer.sign(item))

In [ ]:
import stackstac

ds = stackstac.stack(
    [x.to_dict() for x in signed_items],
    assets=["B04", "B03", "B02"],
    epsg=32721
)
ds

In [ ]:
import stackstac
import rasterio.enums

ds = stackstac.stack(
    [x.to_dict() for x in signed_items],
    assets=["B04", "B03", "B02"],
    epsg=32721,
    bounds_latlon=bbox,
    resolution=100,
    resampling=rasterio.enums.Resampling.bilinear
).where(lambda x: x > 0)
ds

In [ ]:
monthly_rgb = ds.resample(time="MS").median(dim="time")
monthly_rgb

In [ ]:
rgb = monthly_rgb.compute()

In [ ]:
rgb.plot.imshow(row="time", rgb="band", robust=True, size=6);